출처:https://www.kaggle.com/code/ashishpatel26/smote-with-model-lightgbm

### lightgbm(Light Gradient Boosting Machine):
 기계 학습을위한 무료 오픈 소스 분산 경사하강법(Gradient Boostiong) 프레임 워크
### tqdm
 실행 시간이 긴 코드에 대한 진행상황을 확인 가능하도록 하는 라이브러리
### dask
 병렬컴퓨팅을 위한 라이브러리

### dask.dataframe을 import 할 때 오류가 발생할 경우 아래 코드 실행
```
pip install dask[dataframe] --upgrade
```

In [16]:
import warnings
import itertools
import numpy as np 
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, log_loss
import dask
import dask.dataframe as dd
import gc

from yellowbrick.text import TSNEVisualizer

%matplotlib inline
plt.style.use("fivethirtyeight")

import os
print(os.listdir('/content/drive/MyDrive/Colab Notebooks/data_analysis/reducing-commercial-aviation-fatalities'))

warnings.simplefilter(action='ignore')
sns.set_style('whitegrid')

['sample_submission.csv', 'test.csv', 'train.csv']


데이터 불러오기

In [17]:
def gc():
    print(gc.collect())


dtypes = {"crew": "int8",
          "experiment": "category",
          "time": "float32",
          "seat": "int8",
          "eeg_fp1": "float32",
          "eeg_f7": "float32",
          "eeg_f8": "float32",
          "eeg_t4": "float32",
          "eeg_t6": "float32",
          "eeg_t5": "float32",
          "eeg_t3": "float32",
          "eeg_fp2": "float32",
          "eeg_o1": "float32",
          "eeg_p3": "float32",
          "eeg_pz": "float32",
          "eeg_f3": "float32",
          "eeg_fz": "float32",
          "eeg_f4": "float32",
          "eeg_c4": "float32",
          "eeg_p4": "float32",
          "eeg_poz": "float32",
          "eeg_c3": "float32",
          "eeg_cz": "float32",
          "eeg_o2": "float32",
          "ecg": "float32",
          "r": "float32",
          "gsr": "float32",
          "event": "category",
         }

def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else: 
        print('Confusion matrix, without normalization')
    
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max()/2
    for i, j in itertools,product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i,format(cm[i,j],fmt),
                 gorizontalalignment="center",
                 color='white' if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [25]:
%%time
train = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_analysis/reducing-commercial-aviation-fatalities/train.csv',
                    blocksize = 256e6, dtype=dtypes)

CPU times: user 30.6 ms, sys: 23.5 ms, total: 54.2 ms
Wall time: 59.4 ms


In [29]:
%%time
test = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_analysis/reducing-commercial-aviation-fatalities/test.csv',
                    blocksize = 256e6, dtype=dtypes)
# test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_analysis/reducing-commercial-aviation-fatalities/test.csv',dtype=dtypes)

CPU times: user 115 ms, sys: 79.8 ms, total: 195 ms
Wall time: 183 ms


In [27]:
%%time
train = train.compute()
print("Training shape : ", train.shape)

Training shape :  (4867421, 28)
CPU times: user 41.2 s, sys: 1.45 s, total: 42.6 s
Wall time: 32.9 s


In [30]:
%%time
test = test.compute()
print("Testing shape : ", test.shape)
# print(gc.collect())

Testing shape :  (17965143, 28)
CPU times: user 2min 31s, sys: 16.2 s, total: 2min 48s
Wall time: 2min 26s


In [31]:
train_df = train.copy()
test_df = test.copy()

In [32]:
import gc
gc.collect()

1320

In [33]:
test_id = test_df['id']

dic = {'A': 0,'B':1,'C': 2, 'D':3}
dic1 = {'CA':0, 'DA':1, 'SS':3, 'LOFT':4}
train_df['event'] = train_df['event'].apply(lambda x:dic[x])
train_df['event'] = train_df['event'].astype('int8')
train_df['experiment'] = train_df['experiment'].apply(lambda x:dic1[x])
test_df['experiment'] = test_df['experiment'].apply(lambda x:dic1[x])

train_df['experiment'] = train_df['experiment'].astype('int8')
test_df['exdperiment'] = test_df['experiment'].astype('int8')

y = train_df['event']

In [34]:
tr_col = train_df.columns

In [35]:
test_df.head()

,id,crew,experiment,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,...,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr,exdperiment
0,0,1,4,0.000000,0,17.899500,6.127830,0.994807,-28.206200,-47.695499,...,-14.405100,-4.03384,-0.393799,31.83810,17.075600,-8.137350,-7323.120117,643.177002,594.778992,4
1,1,1,4,0.000000,1,45.883202,94.749001,23.290800,1.392000,2.060940,...,-215.179001,2.11832,7.340940,9.66785,1169.239990,-0.024414,-3335.080078,826.724976,136.235001,4
2,2,1,4,0.003906,0,33.120098,28.356501,-7.239220,-7.690860,-25.833799,...,-10.363600,10.95050,6.513360,36.01860,24.356600,0.956158,-7323.120117,643.177002,594.778992,4
3,3,1,4,0.003906,1,43.280102,95.887001,18.702299,-1.432890,-4.232600,...,-214.223007,-4.91354,3.504520,3.87811,1159.400024,-0.047607,-3335.080078,826.724976,136.235001,4
4,4,1,4,0.007812,0,7.929110,3.460380,-10.860800,-26.366699,-25.894699,...,-20.788799,-3.61418,-7.539830,3.08397,13.647900,1.473720,-7323.120117,643.177002,594.778992,4
